In [1]:
import sys
from collections import defaultdict
from textblob import TextBlob
from textblob import Word
import nltk

In [2]:
Term = ["username","text","tweetid"]#该三项为term的主要信息
postings = defaultdict(dict)#建立posting list

In [3]:
#令牌化排序
def tokenize_(document):
    document = document.lower()
    a = document.index("username")#返回指定的索引名称
    b = document.index("clusterno")
    c = document.rindex("tweetid")-1
    d = document.rindex("errorcode")
    e = document.index("text")
    f = document.index("timestr")-3#获取时间戳

    document = document[c:d] + document[a:b] + document[e:f]
    #简化，将tweetid、username、text作为主要信息
    terms = TextBlob(document).words.singularize()#将名词变为单数
    result=[]
    for word in terms:
        except_str = Word(word)
        except_str = except_str.lemmatize("v")#参数为v表示处理动词
        #lemmatize()函数用于对单词进行词性还原，名词为单数形式，动词为原型
        if except_str not in Term:
            result.append(except_str)#添加处理后的term
    return result

In [4]:
#建立posting list
def get_posting():
    global postings
    f = open("./tweets.txt")#使用open()函数读取文档
    lines = f.readlines()#lines中包含文档的每一行
    for line in lines:
        line = tokenize_(line)#将每一行进行词条化
        tweetid = line[0]
        line.pop(0)#去除tweetid，保留username和text
        unique_terms = set(line)#set保持各异性
        for te in unique_terms:
            if te in postings.keys():#若在posting list中
                postings[te].append(tweetid)
            else:
                postings[te] = [tweetid]


In [5]:
#and操作，需要同时存在term1和term2
def op_and(term1, term2):
    global postings
    answer=[]
    if(term1 not in postings) or (term2 not in postings):
        return answer#只要任一不存在则返回空
    else:
        i = len(postings[term1])
        j = len(postings[term2])#获取term1和term2的长度
        x=0
        y=0
        while x<i and y<j:#在二者的posting list中相比较，若对应位置相同，则共同加1
            if postings[term1][x] == postings[term2][y]:
                answer.append(postings[term1][x])
                x+=1
                y+=1
            elif postings[term1][x]<postings[term2][y]:
                x+=1
            else:
                y+=1
        return answer


In [6]:
#or操作，只需要包含其一即可
def op_or(term1,term2):
    answer=[]
    
    if(term1 not in postings) and (term2 not in postings):
        answer = []#如果二者都不在则返回空
    elif term2 not in postings:
        answer = postings[term1]
    elif term1 not in postings:#如果其一不存在，则返回另一个的posing list
        answer = postings[term2]
    else:#二者都存在，以A为准，在B中找A没有的
        answer = postings[term1]
        for item in postings[term2]:
            if item not in answer:
                answer.append(item)
    """  
    if(term1 not in postings) and (term2 not in postings):
        answer=[]
    else:
      
    x=0
    y=0
    i = len(postings[term1])
    j = len(postings[term2])
    while x<i and y<j:
        if postings[term1][x] == postings[term2][y]:
            answer.append(postings[term1][x])
            x+=1
            y+=1
        elif postings[term1][x]<postings[term2][y]:
            answer.append(postings[term1][x])
            x+=1
        else:
            answer.append(postings[term2][y])
            y+=1 
    """
    return answer


In [7]:
#not操作，term1 not term2，需要包含term1，不包含term2
def op_not(term1,term2):
    answer=[]
    if term1 not in postings:#没有term1，直接返回空
        return answer
    elif term2 not in postings:#若没有term2，直接返回term1的posting list
        answer = postings[term1]
        return answer
    else:
        answer = postings[term1]#以term1的posing list为准
        res=[]
        for ans in answer:#若不在term2的posting list中，则符合条件
            if ans not in postings[term2]:
                res.append(ans)
        return res


In [8]:
#将查询语句词条化
def tokenize_query(doc):
    doc=doc.lower()
    terms = TextBlob(doc).words.singularize()#名词变为单数
    res=[]
    for word in terms:
        except_str = Word(word)
        except_str = except_str.lemmatize("v")#动词恢复原形
        res.append(except_str)
    return res


In [9]:
def do_search():
    terms = tokenize_query(input("please input your search query>> "))
    if terms ==[]:
        sys.exit()
    if len(terms)==3:#查询语句只包含三个单词
        if terms[1] == "and":#表示and操作
            answer = op_and(terms[0], terms[2])
            print(answer)
        elif terms[1] == "or":#表示or操作
            answer = op_or(terms[0], terms[2])
            print(answer)
        elif terms[1]=="not":#表示not操作
            answer = op_not(terms[0], terms[2])
            print(answer)
        else:
            print("Error! Please input correctly")
    else:
        print("Input Error, please input like A xxx B")


In [ ]:
def main():
    get_posting()
    while True:
        do_search()

if __name__=="__main__":
    main()